In [1]:
import os
from langchain.chains import ConversationalRetrievalChain
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from tqdm import tqdm
from langchain_openai import OpenAIEmbeddings
os.environ["OPENAI_API_KEY"] = ""


In [21]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.0 MB/s eta 0:00:00


In [2]:
documents = []
c=0
dir_path = './content/docs/'
# Create a List of Documents from all of our files in the ./docs folder
for file in tqdm(os.listdir(dir_path)):

  try:
    pdf_path = dir_path + file
    loader = PyPDFLoader(pdf_path)
    documents.extend(loader.load())
  except:
    c=c+1

 18%|██████████████▏                                                                  | 27/154 [00:49<03:04,  1.45s/it]invalid pdf header: b'<!DOC'
EOF marker not found
 36%|████████████████████████████▉                                                    | 55/154 [01:26<02:17,  1.38s/it]invalid pdf header: b'<!DOC'
EOF marker not found
 77%|█████████████████████████████████████████████████████████████▎                  | 118/154 [03:06<00:48,  1.35s/it]invalid pdf header: b'<!DOC'
EOF marker not found
100%|████████████████████████████████████████████████████████████████████████████████| 154/154 [04:00<00:00,  1.56s/it]


In [4]:
documents

[Document(page_content="Page 1 / 5\n \nAbuse and Domestic\nViolence\nImage credit: Adobe Stock\nSummary:  Abuse is behaviour used to intimidate, isolate, dominate or control another person.\nIf you are being abused, know that abuse is never your fault, and it is never justiﬁed. Things\ndon't have to be this way. Help is available, whether through speaking with your physician, or\ncalling a telephone crisis line, or a woman's shelter.\nWhat is Abuse and Domestic Violence?\nAbuse is behaviour used to intimidate, isolate, dominate or control another person. It may be a pattern of\nbehaviour or it may be a single incident. Abusive behaviour might involve acts or words or even neglect.\n \nAbuse can be physical, sexual, emotional, psychological or ﬁnancial:\nPhysical abuse is when someone hurts you physically in any way. It includes hitting, kicking,\nslapping, pinching, cutting, stabbing, and shooting. All of these examples are assault , which is a crime in\nCanada and the United States.\n

In [29]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 44.

## This cell is if the data has not been loaded to disk yet:

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

vectordb = Chroma.from_documents(all_splits, embedding=OpenAIEmbeddings(), persist_directory="./document_embeddings")


vectordb.persist()


## This cell is if the data has already been loaded to the disk:

In [2]:

vectordb = Chroma(persist_directory="./document_embeddings", embedding_function=OpenAIEmbeddings())


In [3]:
vectordb

In [4]:
pdf_qa = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo'),
    retriever=vectordb.as_retriever(search_kwargs={'k': 30}),
    return_source_documents=True,
    verbose=False
)

In [53]:
chat_history = []

query = "HI!!!"
result = pdf_qa.invoke(
            {"question": query, "chat_history": chat_history}
        )

In [54]:
result["answer"]

'Hello! How can I assist you today?'

In [55]:
result['source_documents']

[Document(page_content='Introduction', metadata={'page': 0, 'source': './content/docs/Supporting Our Emotionally Sensitive Loved Ones with Intense Emotions.pdf'}),
 Document(page_content='http://creativecommons.org/licenses/by-nc-nd/2.5/ca', metadata={'page': 4, 'source': './content/docs/Disruptive Mood Dysregulation Disorder (DMDD).pdf'}),
 Document(page_content='thttp://creativecommons.org/licenses/by-nc-nd/2.5/ca/', metadata={'page': 4, 'source': './content/docs/Narcolepsy and Cataplexy.pdf'}),
 Document(page_content='http://creativecommons.org/licenses/by-nc-nd/2.5/ca/', metadata={'page': 4, 'source': './content/docs/Abuse and Domestic Violence.pdf'}),
 Document(page_content='http://creativecommons.org/licenses/by-nc-nd/2.5/ca/', metadata={'page': 4, 'source': './content/docs/Anxiety in Children and Youth_ Information for Parents and Caregivers.pdf'}),
 Document(page_content='http://creativecommons.org/licenses/by-nc-nd/2.5/ca/', metadata={'page': 2, 'source': './content/docs/Gay, 

In [56]:
result['source_documents'][2].metadata["source"][15:-4]

'Narcolepsy and Cataplexy'

In [57]:
from collections import Counter
source_counter = Counter([result['source_documents'][i].metadata["source"][15:-4] for i in range(30)])

In [60]:
source_counter.most_common(1)

[('Dialectical Behaviour Therapy (DBT)', 2)]

In [33]:
import re
''.join(e for e in ''.join(x for x in docname.title() if not x.isspace()) if e.isalnum())

'PostTraumaticStressDisorderPtsdAndTraumaInChildrenAndYouthInformationForFamiliesAndCaregivers'

In [32]:
docname = result['source_documents'][2].metadata["source"][15:-4]
''.join(x for x in docname.title() if not x.isspace())

'Post-TraumaticStressDisorder(Ptsd)AndTraumaInChildrenAndYouth_InformationForFamiliesAndCaregivers'

In [66]:
len(result['source_documents'])

30

# Obtain a link to the infosheet web-page:

In [10]:
def get_source_doc_title(result):
    '''
    The 'result' variable is the object returned from the Langchain call. 
    '''
    source_path = result['source_documents'][2].metadata["source"][15:-4] # Doc Path of the file
    pascal_case_source_title = ''.join(x for x in source_path.title() if not x.isspace())
    formatted_source_title = ''.join(e for e in pascal_case_source_title if e.isalnum())
    return formatted_source_title

In [11]:
get_source_doc_title(result)

'AlcoholAndSubstanceUseInYouth'

In [34]:
class UncasedDict(dict):          
    # Case-insensitive dictionary
    def __getitem__(self, key):                                                 
        if isinstance(key, str):                                                
            key = key.lower()                                                   
        return super().__getitem__(key)                                         
                                                                                
    def __setitem__(self, key, value):                                          
        if isinstance(key, str):                                                
            key = key.lower()                                                   
        return super().__setitem__(key, value)     
    def get(self, key, default=None):
        if isinstance(key, str):
            key = key.lower()
        return super().get(key, default)
    def __contains__(self, key):
        if isinstance(key, str):
            key = key.lower()
        return super().__contains__(key)

In [13]:
mapping["Antac"] = 1
print(mapping["aNTac"])

NameError: name 'mapping' is not defined

In [35]:
import csv
mapping = UncasedDict()
with open('mapping.csv', 'rt') as f:
    reader = csv.DictReader(f)
    for row in reader:
        mapping[row["Child Class"]] = row["URL"]

In [46]:
mapping[get_source_doc_title(result)]

'https://www.ementalhealth.ca//Toronto/Post-Traumatic-Stress-Disorder-PTSD-and-Trauma-in-Children-and-Youth-Information-for-Families-and-Caregivers/index.php?m=article&ID=57163'

# Obtain Screening tool Data

In [36]:
import csv
mapping_screening_tool = UncasedDict()
with open('Mapping_Screening_Tool.csv', 'rt') as f:
    reader = csv.DictReader(f)
    for row in reader:
        mapping_screening_tool[row["Child Class"]] = (row["Screening Tool Name"], row["ST URLS"])

In [37]:
[toolname.strip() for toolname in mapping_screening_tool["AlcoholProblems"][0].split("|")], [url.strip() for url in mapping_screening_tool["AlcoholProblems"][1].split("|")]

(['Alcohol Problems (in Women)', 'Alcohol Problems (in Men)'],
 ['https://www.ementalhealth.ca/index.php?m=survey&ID=12',
  'https://www.ementalhealth.ca/index.php?m=survey&ID=4'])

In [38]:
mapping_screening_tool.get("aLcoholproblems")

('Alcohol Problems (in Women)    |    Alcohol Problems (in Men)',
 'https://www.ementalhealth.ca/index.php?m=survey&ID=12  |  https://www.ementalhealth.ca/index.php?m=survey&ID=4')

In [41]:
"aLcoholpRoblems" in mapping_screening_tool

True

In [45]:
get_source_doc_title(result)

'PostTraumaticStressDisorderPtsdAndTraumaInChildrenAndYouthInformationForFamiliesAndCaregivers'

In [35]:
get_source_doc_title(result)

'PostTraumaticStressDisorderPtsdAndTraumaInChildrenAndYouthInformationForFamiliesAndCaregivers'

In [10]:
child_class = result['source_documents'][0].metadata['source']

In [12]:
child_class = child_class[14:-4]

In [13]:
child_class

'/Anxiety in Children and Youth_ Information for Parents and Caregivers'

In [14]:
import pandas as pd

In [15]:
mapping = pd.read_csv('Mapping.csv')

In [16]:
mapping.head(100)

,Unnamed: 0,Child Class,Parent Class,URL
0,0,AlcoholProblems,"Addictions (including Drugs, Alcohol and Gambl...",https://www.ementalhealth.ca//Toronto/Alcohol-...
1,1,AlcoholandSubstanceUseinYouth,"Addictions (including Drugs, Alcohol and Gambl...",https://www.ementalhealth.ca//Toronto/Alcohol-...
2,2,ProblemGambling,"Addictions (including Drugs, Alcohol and Gambl...",https://www.ementalhealth.ca//Toronto/Problem-...
3,3,eCigarettesandVaping,"Addictions (including Drugs, Alcohol and Gambl...",https://www.ementalhealth.ca//Toronto/e-Cigare...
4,4,Anger,Anger,https://www.ementalhealth.ca//Toronto/Anger/in...
...,...,...,...,...
95,95,MentalIllnessInformationforSiblings,Mental Health in General,https://www.ementalhealth.ca//Toronto/Mental-I...
96,96,SupportingOurEmotionallySensitiveLovedOneswith...,Mental Health in General,https://www.ementalhealth.ca//Toronto/Supporti...
97,97,Mindfulness101TheBasics,Mindfulness and Mindfulness Therapies,https://www.ementalhealth.ca//Toronto/Mindfuln...
98,98,Mindfulness201BringingMindfulnesstoParenting,Mindfulness and Mindfulness Therapies,https://www.ementalhealth.ca//Toronto/Mindfuln...


In [17]:
# Assuming 'mapping' is your DataFrame
mapping.iloc[9, mapping.columns.get_loc('Child Class')] = child_class


In [18]:
result_row = mapping[mapping['Child Class'] == child_class]


In [19]:
result_row

,Unnamed: 0,Child Class,Parent Class,URL
9,9,/Anxiety in Children and Youth_ Information fo...,Anxiety,https://www.ementalhealth.ca//Toronto/Anxiety-...


In [21]:
df = pd.read_csv('Official_Testing_Data_Clean.csv')



In [58]:
test_file

In [59]:
df = test_file.parse(test_file.sheet_names[0])


In [22]:
df

,Prompt,Provided Heading,Mapped Parent Class
0,How do I know if my child is anxious?,Anxiety,Anxiety
1,What are signs of anxiety in a child?,Anxiety,Anxiety
2,What are signs of anxiety in a child?,Anxiety,Anxiety
3,How do I talk to my child about their anxiety?,Anxiety,Anxiety
4,Can anxiety in children manifest as physical s...,Anxiety,Anxiety
...,...,...,...
382,My daughter’s teeth are decaying from repeated...,Bulimia Nervosa,Eating Disorders including Anorexia and Bulimia
383,My daughter’s teeth are decaying from repeated...,Body Dysmorphic Disorder,Anxiety
384,how do I know if my child is anxious,Anxiety,Anxiety
385,What are signs of anxiety?,Anxiety,Anxiety


In [23]:
df_test_transformed  = df.groupby("Prompt")["Mapped Parent Class"].apply(set)


In [24]:
df_test_transformed

Prompt
Addressing my child's sadness                                                                                     {Depression}
Are there books or resources to help children understand and cope with anxiety?                                      {Anxiety}
Are there certain behaviors that suggest my child could be autistic?                    {Autism and Autism Spectrum Disorders}
Are there effective coping strategies for kids with anxiety?                                                         {Anxiety}
Are there specific traits or characteristics that signal autism in children?            {Autism and Autism Spectrum Disorders}
                                                                                                        ...                   
What signs indicate that my child's sadness may require professional intervention?                                {Depression}
What to do if my child is feeling down?                                                                 

In [25]:
X_test_official = df_test_transformed.index
y_test_official = df_test_transformed.values
print(X_test_official.shape, y_test_official.shape)

(144,) (144,)


In [26]:
predictions = []
from tqdm import tqdm
for j in tqdm(X_test_official):
    result = pdf_qa.invoke(
            {"question": j, "chat_history": chat_history}
        )
    child_class = result['source_documents'][0].metadata['source'][14:-4]
    #print(child_class)
    result_row = mapping[mapping['Child Class'] == child_class]

    predictions.append(result_row['Parent Class'].values)



  6%|█████▏                                                                            | 9/144 [00:23<05:47,  2.58s/it]


KeyboardInterrupt: 

In [65]:
predictions

[array(['Resiliency and Coping'], dtype=object),
 array(['Pandemic (e.g. COVID/Coronavirus), Disasters and Related Emergencies'],
       dtype=object),
 array(['Autism and Autism Spectrum Disorders'], dtype=object),
 array(['Anxiety'], dtype=object),
 array(['Autism and Autism Spectrum Disorders'], dtype=object),
 array(['Anxiety'], dtype=object),
 array(['Autism and Autism Spectrum Disorders'], dtype=object),
 array(['Resiliency and Coping'], dtype=object),
 array(['Pandemic (e.g. COVID/Coronavirus), Disasters and Related Emergencies',
        'School Refusal (and School Phobia)'], dtype=object),
 array(['Resiliency and Coping'], dtype=object),
 array(['Attention Deficit Disorders (ADD/ADHD)'], dtype=object),
 array(['Anxiety'], dtype=object),
 array(['Anxiety'], dtype=object),
 array(['Anxiety'], dtype=object),
 array(['Fetal Alcohol and Fetal Alcohol Spectrum Disorders (FASD)'],
       dtype=object),
 array(['Self-harm including Self-cutting'], dtype=object),
 array(['Anxiety'], dty

In [66]:
X_test_official[22]

'I can’t sleep because of stress and anxiety\xa0'

In [67]:
result = pdf_qa.invoke(
            {"question": X_test_official[22], "chat_history": chat_history}
        )

In [68]:
result['answer']

'Learning how to manage stress in your life may help you sleep better. Consider relaxation training or techniques that can help reduce anxiety or tension, allowing you to fall asleep easier. You can also visit the American Academy of Sleep Medicine website for more information on nightmares and sleep disorders. If your stress and anxiety are impacting your sleep, it might be helpful to discuss this with your healthcare provider for personalized advice and support.'

In [69]:
result['source_documents']

[Document(page_content='Page 4 / 4\nStress management . Learning how to manage stress in your life may help. Consider learning relaxation\ntraining or medication that you are able to reduce the anxiety or tension that keeps you from falling asleep,\nso you can go back to sleep after a nightmare.”\nFor More Information\nThe American Academy of Sleep Medicine has an excellent website with more information about nightmares.\nhttp://www.sleepeducation.com/sleep-disorders-by-category/parasomnias/nightmares/overview\nReferences\nAurora RN, Zak RS, Auerbach SH, Casey KR, Chowdhuri S, Karippot A, Maganti RK, Ramar K, Kristo DA, Bista SR,\nLamm CI, Morgenthaler TI; Standards of Practice Committee; American Academy of Sleep Medicine. Best practice\nguide for the treatment of nightmare disorder in adults. J Clin Sleep Med. 2010 Aug 15;6(4):389-401. PMID:\n20726290; PMCID: PMC2919672.\nAuthors\nWritten by members of the Mental Health Information Committee of the Children’s Hospital of Eastern Onta

In [70]:
child_class = result['source_documents'][0].metadata['source'][14:-4]
result_row = mapping[mapping['Child Class'] == child_class]

In [71]:
result_row['URL']

166    https://www.ementalhealth.ca//Toronto/Nightmar...
Name: URL, dtype: object

In [72]:
def set_accuracy(pred_labels, real_labels_set):
  '''
  pred_labels are the predicted labels (each potentially wrapped in a numpy array)
  real_labels_set are the set of correct labels for each input
  '''
  results = []
  for pred, real_set in zip(pred_labels, real_labels_set):
    # Assuming `pred` is an array with a single value, extract this value
    pred_value = pred.item() if hasattr(pred, 'item') else pred
    if pred_value in real_set:
      results.append(1)
    else:
      results.append(0)

  return sum(results)/len(results)


In [73]:
y_test_official = list(y_test_official)

In [74]:
type(y_test_official[0])

set

In [75]:
set_accuracy(predictions,y_test_official)

ValueError: can only convert an array of size 1 to a Python scalar

In [76]:
len(predictions)

144

In [77]:
c = 0
for j in range(len(predictions)):
    if list(predictions[j])[0] in y_test_official[j]:

        c+=1
    else :print(list(predictions[j])[0],y_test_official[j])

Resiliency and Coping {'Depression'}
Pandemic (e.g. COVID/Coronavirus), Disasters and Related Emergencies {'Anxiety'}
Resiliency and Coping {'Autism and Autism Spectrum Disorders'}
Pandemic (e.g. COVID/Coronavirus), Disasters and Related Emergencies {'Depression'}
Resiliency and Coping {'Depression'}
Fetal Alcohol and Fetal Alcohol Spectrum Disorders (FASD) {'Autism and Autism Spectrum Disorders'}
Self-harm including Self-cutting {'Grief and Bereavement', 'Depression', 'Resiliency and Coping'}
Resiliency and Coping {'Depression'}
Obsessive Compulsive Disorder (OCD) {'Anxiety'}
Sleep Problems and Disorders {'Anxiety', 'Resiliency and Coping'}
Anxiety {'Grief and Bereavement', 'Depression', 'Resiliency and Coping'}
Infant and Early Childhood Mental Health (IECMH) {'Anxiety', 'Anger'}
Resiliency and Coping {'Schizophrenia and Psychosis'}
Pandemic (e.g. COVID/Coronavirus), Disasters and Related Emergencies {'Anxiety', 'Obsessive Compulsive Disorder (OCD)'}
Anger {'Autism and Autism Spectru

In [78]:
c

70

In [79]:
c/len(predictions)

0.4861111111111111